This project was created by the need to be able to analyze the sentiment of the comments of the students of a university, such as permissions problems and violation of privacy we cannot share the data, but although to show how it works we kept the code and only changed the dataset

## Python libraries

First of all we start by installing different libraries to be used throughout the project, we use libraries like:

*   pandas and numpy for handling datasets
*   tensorflow and models.official for bert's models
*   sklearn for the use of Kfold and some metrics

In [1]:
import pandas as pd
import numpy as np
import os 

import tensorflow as tf
from tensorflow import keras
strategy = tf.distribute.OneDeviceStrategy(device='/gpu:0')

!wget https://storage.googleapis.com/cloud-tpu-checkpoints/bert/keras_bert/multi_cased_L-12_H-768_A-12.tar.gz
!tar xvzf multi_cased_L-12_H-768_A-12.tar.gz

!pip install -q tf-models-official==2.3.0
from official.nlp.albert import configs as albert_configs
from official.nlp.bert import configs
from official.modeling import tf_utils
from official.nlp.modeling import networks

from sklearn.model_selection import KFold
from sklearn import metrics

--2021-04-27 13:45:29--  https://storage.googleapis.com/cloud-tpu-checkpoints/bert/keras_bert/multi_cased_L-12_H-768_A-12.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.167.128, 64.233.166.128, 74.125.71.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.167.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 660200694 (630M) [application/octet-stream]
Saving to: ‘multi_cased_L-12_H-768_A-12.tar.gz.1’

multi_cased_L-12_H- 100%[===================>] 629.62M  46.7MB/s    in 14s     

2021-04-27 13:45:43 (44.2 MB/s) - ‘multi_cased_L-12_H-768_A-12.tar.gz.1’ saved [660200694/660200694]

./multi_cased_L-12_H-768_A-12/
./multi_cased_L-12_H-768_A-12/vocab.txt
./multi_cased_L-12_H-768_A-12/bert_model.ckpt.index
./multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
./multi_cased_L-12_H-768_A-12/bert_config.json


##The model

For  our  case  study,  BERT  encodes  each  comment  into  a768-dimensional embedding and, then, a dense layer trans-forms  the  embeddings  into  a  three-dimensional  vector  foreach comment that indicates the probability that the com-ment belongs to each of the three classes - “positive”, “neg-ative” or “neutral”.  We adopted the BERT-Base, Multilin-gual Cased version2(for 104 languages, with 12-layer, 768-hidden, 12-heads, 110M parameters), which is required sincethe comments are written in Portuguese.  In order to signif-icantly speed up the training and inference with our model,we  limited  the  size  of  each  input  comment  to  64  tokens,which is enough to cover the vast majority of the comments.Any comment with less than 64 tokens was padded with the‘[PAD]’ symbol already allocated in BERT’s vocabulary andany comment with more than 64 tokens was truncated.

In [2]:
#2
MODEL_DIR = "multi_cased_L-12_H-768_A-12" #@param {type:"string"} ['multi_cased_L-12_H-768_A-12']
MLP_DIMS = "" #@param {type:"string"}
MAX_SEQ_LENGTH = 64 #@param {type:"integer"}
N_CLASSES = 3 #@param {type:"integer"}

if MLP_DIMS:
  mlp_dims = MLP_DIMS.split(",")
  mlp_dims = [ int(x) for x in mlp_dims ]
  print(mlp_dims)
else:
  mlp_dims = []

config_file_path = "/content/{}/bert_config.json".format(MODEL_DIR)
bert_config = configs.BertConfig.from_json_file(config_file_path)

#if flag == 1 then path of BERT and checkpoint.restore
#if flag == 2 then path of BERT without checkpoint.restore
#if flag == 3 then path for our checkpoint created in cell 7
def create_model(flag):
  if flag == 3:
    checkpoint_path = "pre_trained_checkpoint/cp.ckpt"
  else:
    checkpoint_path = "/content/{}/bert_model.ckpt".format(MODEL_DIR)

  cfg = bert_config
  print(cfg)

  kwargs = dict(
        vocab_size=cfg.vocab_size,
        hidden_size=cfg.hidden_size,
        num_layers=cfg.num_hidden_layers,
        num_attention_heads=cfg.num_attention_heads,
        intermediate_size=cfg.intermediate_size,
        activation=tf_utils.get_activation(cfg.hidden_act),
        dropout_rate=cfg.hidden_dropout_prob,
        attention_dropout_rate=cfg.attention_probs_dropout_prob,
        sequence_length=MAX_SEQ_LENGTH,
        max_sequence_length=cfg.max_position_embeddings,
        type_vocab_size=cfg.type_vocab_size,
        embedding_width=cfg.embedding_size,
        initializer=tf.keras.initializers.TruncatedNormal(
            stddev=cfg.initializer_range))

  bert_encoder = networks.TransformerEncoder(**kwargs)

  checkpoint = tf.train.Checkpoint(model=bert_encoder)
  options = tf.train.CheckpointOptions(experimental_io_device="/job:localhost")
  if flag == 1:
    checkpoint.restore(checkpoint_path, options=options).assert_consumed().run_restore_ops()
  
  word_ids_input = tf.keras.layers.Input(
      shape=(MAX_SEQ_LENGTH,), dtype=tf.int32, name='word_ids')
  mask_input = tf.keras.layers.Input(
      shape=(MAX_SEQ_LENGTH,), dtype=tf.int32, name='masks')
  type_ids_input = tf.keras.layers.Input(
      shape=(MAX_SEQ_LENGTH,), dtype=tf.int32, name='type_ids')

  sequence_output, pooled_output = bert_encoder(
        [word_ids_input, mask_input, type_ids_input])
  
  print(sequence_output)
  print(pooled_output)
  
  if mlp_dims:
    mlp = tf.keras.layers.Dense(mlp_dims[0], activation='tanh')(pooled_output)
    for dims in mlp_dims[1:]:
      mlp = tf.keras.layers.Dense(dims, activation='tanh')(mlp)
  else:
    mlp = pooled_output

  output = tf.keras.layers.Dense(N_CLASSES, activation='softmax')(mlp)

  model = tf.keras.Model(
      inputs=[word_ids_input, mask_input, type_ids_input], outputs=output)
  return model

In [3]:
#3
MODEL_DIR = "multi_cased_L-12_H-768_A-12" #@param {type:"string"} ['multi_cased_L-12_H-768_A-12']

from official.nlp.bert import tokenization

vocab_file = "/content/{}/vocab.txt".format(MODEL_DIR)

tokenizer = tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=False)

#tokenizer.tokenize("Este é um exemplo de utilização de um BERT tokenizer em português.")

In [4]:
#4
def transform_to_dataset(input_text, labels):
  def generator():
    for i, (key, text) in enumerate(input_text.iteritems()):
      label = labels[key]
      unicode_text = tokenization.convert_to_unicode(text)
      tokens = tokenizer.tokenize(unicode_text)
      tokens = ['[CLS]'] + tokens + ['[SEP]']
      token_ids = tokenizer.convert_tokens_to_ids(tokens)

      mask_ids = []
      type_ids = []
      for i in range(len(token_ids)):
        mask_ids.append(1)
        type_ids.append(0)

      pad_id = tokenizer.convert_tokens_to_ids(['[PAD]'])[0]

      if len(token_ids) > MAX_SEQ_LENGTH:
        token_ids = token_ids[:MAX_SEQ_LENGTH]
        tokens = tokens[:MAX_SEQ_LENGTH]
        mask_ids = mask_ids[:MAX_SEQ_LENGTH]
        type_ids = type_ids[:MAX_SEQ_LENGTH]
      else:
        for _ in range(MAX_SEQ_LENGTH - len(token_ids)):
          token_ids.append(pad_id)
          tokens.append('[PAD]')
          mask_ids.append(0)
          type_ids.append(1)

      yield {"word_ids": token_ids,
             "masks": mask_ids, "type_ids": type_ids}, label
  return generator

## The Dataset
As we mentioned at the beginning we will not be using our dataset due to confidentiality problems, but for this example we will be using a public dataset with the product comments. [B2W-Reviews01](https://github.com/b2wdigital/b2w-reviews01)

As our data is divided into 800 comments that were manually classified to know their sentiment, the dataset that we used to train had 10077 comments, we divided the product dataset into several datasets but keeping the amount of comments that we used for each one.

In [5]:
#5
df = pd.read_csv("https://raw.githubusercontent.com/b2wdigital/b2w-reviews01/master/B2W-Reviews01.csv", sep = ";")

#New feature "label" - 0:'Negativo, 1:'Neutro' and 2:'Positivo' 
df['label'] = np.where(df['overall_rating'] >= 4, 2, np.where(df['overall_rating'] <= 2, 0, 1))

df_annotatedComments = df[:800]
df_trainBERT = df[800:10877]
df_notAnnotatedComments = df[10877:]


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Pre-Train

We started our experimental setup by executing a pre-trainingstep that aims at getting the model used to the style of comments through non-annotated data.  We started  the  pre-training  experiment  with  the  multilingual BERT checkpoint that is publicly available and trained for 10 epochs, resulting in a newly trained checkpoint which we call from this point on the pre-trained checkpoint.

In [6]:
#6 
input_text_train = df_trainBERT['review_text']
label_train = df_trainBERT['label']

train_dataset_na = tf.data.Dataset.from_generator(
    transform_to_dataset(input_text_train, label_train),
    output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
    output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))

print(train_dataset_na)
train_dataset_na = train_dataset_na.batch(32)

<FlatMapDataset shapes: ({word_ids: (64,), masks: (64,), type_ids: (64,)}, ()), types: ({word_ids: tf.int64, masks: tf.int64, type_ids: tf.int64}, tf.int64)>


In [7]:
#7
STEPS_PER_EPOCH = -1 #@param {type:"integer"}
VALIDATION_STEPS = -1 #@param {type:"integer"}

steps_per_epoch = STEPS_PER_EPOCH
if STEPS_PER_EPOCH <= 0:
  steps_per_epoch = None

validation_steps = VALIDATION_STEPS
if VALIDATION_STEPS <= 0:
  validation_steps = None

checkpoint_path = "pre_trained_checkpoint/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

with strategy.scope():
  #model = tf.keras.models.clone_model(local_model)
  model = create_model(1)
  model.summary()
  # Dica: o learning rate com o BERT tem sempre de ser muito pequeno, caso contrário, ele vai esquecer o pre-treino durante os primeiros passos.
  opt = keras.optimizers.Adam(learning_rate=0.00001)
  model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
  history = model.fit(
        train_dataset_na,
        epochs=10, steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps, callbacks=[cp_callback])

KerasTensor(type_spec=TensorSpec(shape=(None, 64, 768), dtype=tf.float32, name=None), name='transformer_encoder/transformer/layer_11/output_layer_norm/batchnorm/add_1:0', description="created by layer 'transformer_encoder'")
KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='transformer_encoder/pooler_transform/Tanh:0', description="created by layer 'transformer_encoder'")
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
word_ids (InputLayer)           [(None, 64)]         0                                            
__________________________________________________________________________________________________
masks (InputLayer)              [(None, 64)]         0                                            
_____________________________________________________________________________

**THE EXPERIMENTS**

After the pre-training step, we proceeded to experiment with three setups, using a 5-fold cross-validation strategy, applied to the set of 800 manually classified comments. Therefore, each round of cross-validation used 640 comments for training and 160 comments for testing. 

In [6]:
#8 
df_train_a = df_annotatedComments[:640]
df_test_a = df_annotatedComments[640:]

input_text_train = df_train_a['review_text']
label_train = df_train_a['label']
input_text_test = df_test_a['review_text']
label_test = df_test_a['label']

# Merge inputs and targets
inputs = pd.Series(np.concatenate((input_text_train, input_text_test), axis=0))
targets = pd.Series(np.concatenate((label_train, label_test), axis=0))

# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Define per-fold score containers for from-scratch
acc_per_fold_640_fs = []
loss_per_fold_640_fs = []
precision_per_fold_640_fs = []
recall_per_fold_640_fs = []

acc_per_fold_320_fs = []
loss_per_fold_320_fs = []
precision_per_fold_320_fs = []
recall_per_fold_320_fs = []

acc_per_fold_160_fs = []
loss_per_fold_160_fs = []
precision_per_fold_160_fs = []
recall_per_fold_160_fs = []

acc_per_fold_80_fs = []
loss_per_fold_80_fs = []
precision_per_fold_80_fs = []
recall_per_fold_80_fs = []

acc_per_fold_40_fs = []
loss_per_fold_40_fs = []
precision_per_fold_40_fs = []
recall_per_fold_40_fs = []

f1_fold_640_fs = []
f1_fold_320_fs = []
f1_fold_160_fs = []
f1_fold_80_fs = []
f1_fold_40_fs = []

# Define per-fold score containers for fine-tuned
acc_per_fold_640_ft = []
loss_per_fold_640_ft = []
precision_per_fold_640_ft = []
recall_per_fold_640_ft = []

acc_per_fold_320_ft = []
loss_per_fold_320_ft = []
precision_per_fold_320_ft = []
recall_per_fold_320_ft = []

acc_per_fold_160_ft = []
loss_per_fold_160_ft = []
precision_per_fold_160_ft = []
recall_per_fold_160_ft = []

acc_per_fold_80_ft = []
loss_per_fold_80_ft = []
precision_per_fold_80_ft = []
recall_per_fold_80_ft = []

acc_per_fold_40_ft = []
loss_per_fold_40_ft = []
precision_per_fold_40_ft = []
recall_per_fold_40_ft = []

f1_fold_640_ft = []
f1_fold_320_ft = []
f1_fold_160_ft = []
f1_fold_80_ft = []
f1_fold_40_ft = []

## Zero-Shot

This experiment does not perform any training with the manually classified comments.  Instead, it performs inference directly using the pre-trained checkpoint that resulted from the pre-training step on the test set.  If this model’s performance was good,  then it  would  show  that  manually  annotating  comments would not be necessary.

In [9]:
#9

STEPS_PER_EPOCH = -1 #@param {type:"integer"}
VALIDATION_STEPS = -1 #@param {type:"integer"}

steps_per_epoch = STEPS_PER_EPOCH
if STEPS_PER_EPOCH <= 0:
  steps_per_epoch = None

validation_steps = VALIDATION_STEPS
if VALIDATION_STEPS <= 0:
  validation_steps = None

# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []
precision_per_fold = []
recall_per_fold = []

with strategy.scope():
  # K-fold Cross Validation model evaluation
  fold_no = 1
  for train, test in kfold.split(inputs, targets):
    test_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[test], targets[test]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    test_dataset = test_dataset.batch(32)

    model = create_model(3)
    checkpoint_path = "pre_trained_checkpoint/cp.ckpt"
    model.load_weights(checkpoint_path)
    model.summary() 
    opt = keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
    history_zeroShot = model.evaluate(
        test_dataset,
        batch_size=10, steps=10)
    
    scores = model.evaluate(test_dataset, verbose=0)
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    predictions = model.predict(test_dataset)
    matrix = metrics.confusion_matrix(targets[test],predictions.argmax(1))
    #print(matrix)
    recall_per_fold.append(np.mean(np.diag(matrix) / np.sum(matrix, axis = 1))*100)
    precision_per_fold.append(np.mean(np.diag(matrix) / np.sum(matrix, axis = 0))*100)

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}% - Precision: {precision_per_fold[i]} - Recall: {recall_per_fold[i]}')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print(f'> Precision: {np.mean(precision_per_fold)}(+- {np.std(precision_per_fold)})')
print(f'> Recall: {np.mean(recall_per_fold)}(+- {np.std(recall_per_fold)})')
print('------------------------------------------------------------------------') 

KerasTensor(type_spec=TensorSpec(shape=(None, 64, 768), dtype=tf.float32, name=None), name='transformer_encoder_1/transformer/layer_11/output_layer_norm/batchnorm/add_1:0', description="created by layer 'transformer_encoder_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='transformer_encoder_1/pooler_transform/Tanh:0', description="created by layer 'transformer_encoder_1'")
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
word_ids (InputLayer)           [(None, 64)]         0                                            
__________________________________________________________________________________________________
masks (InputLayer)              [(None, 64)]         0                                            
___________________________________________________________________

## From  scratch  

This experiment does not use the pre-trained checkpoint that resulted from the pre-training step. Instead,  it  starts  with  the  multilingual  BERT checkpoint and uses the manually classified comments to train and evaluate the model. The objective of this experiment is to understand if the pre-training step is necessary to obtain top-quality results.

In [10]:
#10

STEPS_PER_EPOCH = -1 #@param {type:"integer"}
VALIDATION_STEPS = -1 #@param {type:"integer"}

steps_per_epoch = STEPS_PER_EPOCH
if STEPS_PER_EPOCH <= 0:
  steps_per_epoch = None

validation_steps = VALIDATION_STEPS
if VALIDATION_STEPS <= 0:
  validation_steps = None

with strategy.scope():
  # K-fold Cross Validation model evaluation
  fold_no = 1
  for train, test in kfold.split(inputs, targets):
    test_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[test], targets[test]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    test_dataset = test_dataset.batch(32)

    #640
    train_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[train], targets[train]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    train_dataset = train_dataset.batch(32)

    model = create_model(1)
    model.summary() 
    opt = keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
    history_root = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=10, steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps)
    
    scores = model.evaluate(test_dataset, verbose=0)
    acc_per_fold_640_fs.append(scores[1] * 100)
    loss_per_fold_640_fs.append(scores[0])

    predictions = model.predict(test_dataset)
    matrix = metrics.confusion_matrix(targets[test],predictions.argmax(1))
    p = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 0))*100)
    r = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 1))*100)
    recall_per_fold_640_fs.append(r)
    precision_per_fold_640_fs.append(p)
    f1_fold_640_fs.append(2*((p*r)/(p+r)))

    #320
    train_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[train[:320]], targets[train[:320]]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    train_dataset = train_dataset.batch(32)

    model = create_model(1)
    model.summary() 
    opt = keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
    history_root = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=10, steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps)
    
    scores = model.evaluate(test_dataset, verbose=0)
    acc_per_fold_320_fs.append(scores[1] * 100)
    loss_per_fold_320_fs.append(scores[0])
    predictions = model.predict(test_dataset)
    matrix = metrics.confusion_matrix(targets[test],predictions.argmax(1))
    p = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 0))*100)
    r = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 1))*100)
    recall_per_fold_320_fs.append(r)
    precision_per_fold_320_fs.append(p)
    f1_fold_320_fs.append(2*((p*r)/(p+r)))

    #160
    train_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[train[:160]], targets[train[:160]]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    train_dataset = train_dataset.batch(32)

    model = create_model(1)
    model.summary() 
    opt = keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
    history_root = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=10, steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps)
    
    scores = model.evaluate(test_dataset, verbose=0)
    acc_per_fold_160_fs.append(scores[1] * 100)
    loss_per_fold_160_fs.append(scores[0])

    predictions = model.predict(test_dataset)
    matrix = metrics.confusion_matrix(targets[test],predictions.argmax(1))
    p = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 0))*100)
    r = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 1))*100)
    recall_per_fold_160_fs.append(r)
    precision_per_fold_160_fs.append(p)
    f1_fold_160_fs.append(2*((p*r)/(p+r)))
    
    #80
    train_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[train[:80]], targets[train[:80]]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    train_dataset = train_dataset.batch(32)

    model = create_model(1)
    model.summary() 
    opt = keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
    history_root = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=10, steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps)
    
    scores = model.evaluate(test_dataset, verbose=0)
    acc_per_fold_80_fs.append(scores[1] * 100)
    loss_per_fold_80_fs.append(scores[0])

    predictions = model.predict(test_dataset)
    matrix = metrics.confusion_matrix(targets[test],predictions.argmax(1))
    p = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 0))*100)
    r = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 1))*100)
    recall_per_fold_80_fs.append(r)
    precision_per_fold_80_fs.append(p)
    f1_fold_80_fs.append(2*((p*r)/(p+r)))

    #40
    train_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[train[:40]], targets[train[:40]]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    train_dataset = train_dataset.batch(32)

    model = create_model(1)
    model.summary() 
    opt = keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
    history_root = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=10, steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps)
    
    scores = model.evaluate(test_dataset, verbose=0)
    acc_per_fold_40_fs.append(scores[1] * 100)
    loss_per_fold_40_fs.append(scores[0])

    predictions = model.predict(test_dataset)
    matrix = metrics.confusion_matrix(targets[test],predictions.argmax(1))
    p = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 0))*100)
    r = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 1))*100)
    recall_per_fold_40_fs.append(r)
    precision_per_fold_40_fs.append(p)
    f1_fold_40_fs.append(2*((p*r)/(p+r)))

KerasTensor(type_spec=TensorSpec(shape=(None, 64, 768), dtype=tf.float32, name=None), name='transformer_encoder_6/transformer/layer_11/output_layer_norm/batchnorm/add_1:0', description="created by layer 'transformer_encoder_6'")
KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='transformer_encoder_6/pooler_transform/Tanh:0', description="created by layer 'transformer_encoder_6'")
Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
word_ids (InputLayer)           [(None, 64)]         0                                            
__________________________________________________________________________________________________
masks (InputLayer)              [(None, 64)]         0                                            
___________________________________________________________________

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:155: RuntimeWarning: invalid value encountered in true_divide


KerasTensor(type_spec=TensorSpec(shape=(None, 64, 768), dtype=tf.float32, name=None), name='transformer_encoder_10/transformer/layer_11/output_layer_norm/batchnorm/add_1:0', description="created by layer 'transformer_encoder_10'")
KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='transformer_encoder_10/pooler_transform/Tanh:0', description="created by layer 'transformer_encoder_10'")
Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
word_ids (InputLayer)           [(None, 64)]         0                                            
__________________________________________________________________________________________________
masks (InputLayer)              [(None, 64)]         0                                            
______________________________________________________________

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:189: RuntimeWarning: invalid value encountered in true_divide


KerasTensor(type_spec=TensorSpec(shape=(None, 64, 768), dtype=tf.float32, name=None), name='transformer_encoder_11/transformer/layer_11/output_layer_norm/batchnorm/add_1:0', description="created by layer 'transformer_encoder_11'")
KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='transformer_encoder_11/pooler_transform/Tanh:0', description="created by layer 'transformer_encoder_11'")
Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
word_ids (InputLayer)           [(None, 64)]         0                                            
__________________________________________________________________________________________________
masks (InputLayer)              [(None, 64)]         0                                            
______________________________________________________________

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:121: RuntimeWarning: invalid value encountered in true_divide


KerasTensor(type_spec=TensorSpec(shape=(None, 64, 768), dtype=tf.float32, name=None), name='transformer_encoder_14/transformer/layer_11/output_layer_norm/batchnorm/add_1:0', description="created by layer 'transformer_encoder_14'")
KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='transformer_encoder_14/pooler_transform/Tanh:0', description="created by layer 'transformer_encoder_14'")
Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
word_ids (InputLayer)           [(None, 64)]         0                                            
__________________________________________________________________________________________________
masks (InputLayer)              [(None, 64)]         0                                            
______________________________________________________________

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in true_divide


KerasTensor(type_spec=TensorSpec(shape=(None, 64, 768), dtype=tf.float32, name=None), name='transformer_encoder_27/transformer/layer_11/output_layer_norm/batchnorm/add_1:0', description="created by layer 'transformer_encoder_27'")
KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='transformer_encoder_27/pooler_transform/Tanh:0', description="created by layer 'transformer_encoder_27'")
Model: "model_27"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
word_ids (InputLayer)           [(None, 64)]         0                                            
__________________________________________________________________________________________________
masks (InputLayer)              [(None, 64)]         0                                            
______________________________________________________________

In [11]:
# == Provide average scores ==
print('-------------------------------640-----------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_640_fs)} (+- {np.std(acc_per_fold_640_fs)})')
print(f'> Loss: {np.mean(loss_per_fold_640_fs)}')
print(f'> F1: {np.mean(f1_fold_640_fs)}(+- {np.std(f1_fold_640_fs)})')
print(f'> Precision: {np.mean(precision_per_fold_640_fs)}(+- {np.std(precision_per_fold_640_fs)})')
print(f'> Recall: {np.mean(recall_per_fold_640_fs)}(+- {np.std(recall_per_fold_640_fs)})')
print('------------------------------------------------------------------------') 

print('-------------------------------320-----------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_320_fs)} (+- {np.std(acc_per_fold_320_fs)})')
print(f'> Loss: {np.mean(loss_per_fold_320_fs)}')
print(f'> F1: {np.mean(f1_fold_320_fs)}(+- {np.std(f1_fold_320_fs)})')
print(f'> Precision: {np.mean(precision_per_fold_320_fs)}(+- {np.std(precision_per_fold_320_fs)})')
print(f'> Recall: {np.mean(recall_per_fold_320_fs)}(+- {np.std(recall_per_fold_320_fs)})')
print('------------------------------------------------------------------------') 

print('-------------------------------160-----------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_160_fs)} (+- {np.std(acc_per_fold_160_fs)})')
print(f'> Loss: {np.mean(loss_per_fold_160_fs)}')
print(f'> F1: {np.mean(f1_fold_160_fs)}(+- {np.std(f1_fold_160_fs)})')
print(f'> Precision: {np.mean(precision_per_fold_160_fs)}(+- {np.std(precision_per_fold_160_fs)})')
print(f'> Recall: {np.mean(recall_per_fold_160_fs)}(+- {np.std(recall_per_fold_160_fs)})')
print('------------------------------------------------------------------------') 

print('-------------------------------80-----------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_80_fs)} (+- {np.std(acc_per_fold_80_fs)})')
print(f'> Loss: {np.mean(loss_per_fold_80_fs)}')
print(f'> F1: {np.mean(f1_fold_80_fs)}(+- {np.std(f1_fold_80_fs)})')
print(f'> Precision: {np.mean(np.nan_to_num(precision_per_fold_80_fs))}(+- {np.std(np.nan_to_num(precision_per_fold_80_fs))})')
print(f'> Recall: {np.mean(recall_per_fold_80_fs)}(+- {np.std(recall_per_fold_80_fs)})')
print('------------------------------------------------------------------------') 

print('-------------------------------40-----------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_40_fs)} (+- {np.std(acc_per_fold_40_fs)})')
print(f'> Loss: {np.mean(loss_per_fold_40_fs)}')
print(f'> F1: {np.mean(f1_fold_40_fs)}(+- {np.std(f1_fold_40_fs)})')
print(f'> Precision: {np.mean(np.nan_to_num(precision_per_fold_40_fs))}(+- {np.std(np.nan_to_num(precision_per_fold_40_fs))})')
print(f'> Recall: {np.mean(recall_per_fold_40_fs)}(+- {np.std(recall_per_fold_40_fs)})')
print('------------------------------------------------------------------------') 

-------------------------------640-----------------------------------------
Average scores for all folds:
> Accuracy: 73.50000023841858 (+- 3.2258713469212923)
> Loss: 1.0759462356567382
> F1: 43.03528317797685(+- 21.52658750175285)
> Precision: 43.38432367505165(+- 21.809583128365226)
> Recall: 53.51387643299665(+- 2.771545853892167)
------------------------------------------------------------------------
-------------------------------320-----------------------------------------
Average scores for all folds:
> Accuracy: 73.49999904632568 (+- 5.598549195947416)
> Loss: 0.908374798297882
> F1: 54.11694125370591(+- 4.210283806003055)
> Precision: 54.25795073468261(+- 5.553665720867327)
> Recall: 54.103147260920935(+- 3.399926957757947)
------------------------------------------------------------------------
-------------------------------160-----------------------------------------
Average scores for all folds:
> Accuracy: 70.37500023841858 (+- 3.273950868833698)
> Loss: 0.9377208828926

## Fine-tuned

This experiment uses the pre-trained checkpoint that resulted from the pre-training step and then uses it as the starting point when training  with the manually classified documents. This experiment aims at evaluating if combining pre-training and manually annotated comments helps in obtaining top-quality results.

In [7]:
#11
#- Fine-tuned: Carregar o checkpoint do pré-treino e treinar durante o mesmo número epochs que a variante de raíz

STEPS_PER_EPOCH = -1 #@param {type:"integer"}
VALIDATION_STEPS = -1 #@param {type:"integer"}

steps_per_epoch = STEPS_PER_EPOCH
if STEPS_PER_EPOCH <= 0:
  steps_per_epoch = None

validation_steps = VALIDATION_STEPS
if VALIDATION_STEPS <= 0:
  validation_steps = None

with strategy.scope():
  # K-fold Cross Validation model evaluation
  fold_no = 1
  for train, test in kfold.split(inputs, targets):
    test_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[test], targets[test]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    test_dataset = test_dataset.batch(32)

    #640
    train_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[train], targets[train]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    train_dataset = train_dataset.batch(32)

    
    model = create_model(3)
    checkpoint_path = "pre_trained_checkpoint/cp.ckpt"
    model.load_weights(checkpoint_path)
    model.summary()
    opt = keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
    history_fineTuned = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=10, steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps)

    scores = model.evaluate(test_dataset, verbose=0)
    acc_per_fold_640_ft.append(scores[1] * 100)
    loss_per_fold_640_ft.append(scores[0])

    predictions = model.predict(test_dataset)
    matrix = metrics.confusion_matrix(targets[test],predictions.argmax(1))
    p = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 0))*100)
    r = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 1))*100)
    recall_per_fold_640_ft.append(r)
    precision_per_fold_640_ft.append(p)
    f1_fold_640_ft.append(2*((p*r)/(p+r)))

    #320
    train_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[train[:320]], targets[train[:320]]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    train_dataset = train_dataset.batch(32)

    model = create_model(3)
    checkpoint_path = "pre_trained_checkpoint/cp.ckpt"
    model.load_weights(checkpoint_path)
    model.summary() 
    opt = keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
    history_fineTuned = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=10, steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps)

    scores = model.evaluate(test_dataset, verbose=0)
    acc_per_fold_320_ft.append(scores[1] * 100)
    loss_per_fold_320_ft.append(scores[0])

    predictions = model.predict(test_dataset)
    matrix = metrics.confusion_matrix(targets[test],predictions.argmax(1))
    p = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 0))*100)
    r = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 1))*100)
    recall_per_fold_320_ft.append(r)
    precision_per_fold_320_ft.append(p)
    f1_fold_320_ft.append(2*((p*r)/(p+r)))

    #160
    train_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[train[:160]], targets[train[:160]]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    train_dataset = train_dataset.batch(32)

    model = create_model(3)
    checkpoint_path = "pre_trained_checkpoint/cp.ckpt"
    model.load_weights(checkpoint_path)
    model.summary()
    opt = keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
    history_fineTuned = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=10, steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps)

    scores = model.evaluate(test_dataset, verbose=0)
    acc_per_fold_160_ft.append(scores[1] * 100)
    loss_per_fold_160_ft.append(scores[0])

    predictions = model.predict(test_dataset)
    matrix = metrics.confusion_matrix(targets[test],predictions.argmax(1))
    p = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 0))*100)
    r = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 1))*100)
    recall_per_fold_160_ft.append(r)
    precision_per_fold_160_ft.append(p)
    f1_fold_160_ft.append(2*((p*r)/(p+r)))

    #80
    train_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[train[:80]], targets[train[:80]]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    train_dataset = train_dataset.batch(32)

    model = create_model(3)
    checkpoint_path = "pre_trained_checkpoint/cp.ckpt"
    model.load_weights(checkpoint_path)
    model.summary() 
    opt = keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
    history_fineTuned = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=10, steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps)

    scores = model.evaluate(test_dataset, verbose=0)
    acc_per_fold_80_ft.append(scores[1] * 100)
    loss_per_fold_80_ft.append(scores[0])

    predictions = model.predict(test_dataset)
    matrix = metrics.confusion_matrix(targets[test],predictions.argmax(1))
    p = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 0))*100)
    r = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 1))*100)
    recall_per_fold_80_ft.append(r)
    precision_per_fold_80_ft.append(p)
    f1_fold_80_ft.append(2*((p*r)/(p+r)))

    #40
    train_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[train[:40]], targets[train[:40]]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    train_dataset = train_dataset.batch(32)

    model = create_model(3)
    checkpoint_path = "pre_trained_checkpoint/cp.ckpt"
    model.load_weights(checkpoint_path)
    model.summary() 
    opt = keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
    history_fineTuned = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=10, steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps)

    scores = model.evaluate(test_dataset, verbose=0)
    acc_per_fold_40_ft.append(scores[1] * 100)
    loss_per_fold_40_ft.append(scores[0])

    predictions = model.predict(test_dataset)
    matrix = metrics.confusion_matrix(targets[test],predictions.argmax(1))
    p = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 0))*100)
    r = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 1))*100)
    recall_per_fold_40_ft.append(r)
    precision_per_fold_40_ft.append(p)
    f1_fold_40_ft.append(2*((p*r)/(p+r)))


KerasTensor(type_spec=TensorSpec(shape=(None, 64, 768), dtype=tf.float32, name=None), name='transformer_encoder/transformer/layer_11/output_layer_norm/batchnorm/add_1:0', description="created by layer 'transformer_encoder'")
KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='transformer_encoder/pooler_transform/Tanh:0', description="created by layer 'transformer_encoder'")
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
word_ids (InputLayer)           [(None, 64)]         0                                            
__________________________________________________________________________________________________
masks (InputLayer)              [(None, 64)]         0                                            
_____________________________________________________________________________

In [8]:
# == Provide average scores ==
print('-------------------------------640-----------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_640_ft)} (+- {np.std(acc_per_fold_640_ft)})')
print(f'> Loss: {np.mean(loss_per_fold_640_ft)}')
print(f'> F1: {np.mean(f1_fold_640_ft)}(+- {np.std(f1_fold_640_ft)})')
print(f'> Precision: {np.mean(precision_per_fold_640_ft)}(+- {np.std(precision_per_fold_640_ft)})')
print(f'> Recall: {np.mean(recall_per_fold_640_ft)}(+- {np.std(recall_per_fold_640_ft)})')
print('------------------------------------------------------------------------') 

print('-------------------------------320-----------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_320_ft)} (+- {np.std(acc_per_fold_320_ft)})')
print(f'> Loss: {np.mean(loss_per_fold_320_ft)}')
print(f'> F1: {np.mean(f1_fold_320_ft)}(+- {np.std(f1_fold_320_ft)})')
print(f'> Precision: {np.mean(precision_per_fold_320_ft)}(+- {np.std(precision_per_fold_320_ft)})')
print(f'> Recall: {np.mean(recall_per_fold_320_ft)}(+- {np.std(recall_per_fold_320_ft)})')
print('------------------------------------------------------------------------') 

print('-------------------------------160-----------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_160_ft)} (+- {np.std(acc_per_fold_160_ft)})')
print(f'> Loss: {np.mean(loss_per_fold_160_ft)}')
print(f'> F1: {np.mean(f1_fold_160_ft)}(+- {np.std(f1_fold_160_ft)})')
print(f'> Precision: {np.mean(precision_per_fold_160_ft)}(+- {np.std(precision_per_fold_160_ft)})')
print(f'> Recall: {np.mean(recall_per_fold_160_ft)}(+- {np.std(recall_per_fold_160_ft)})')
print('------------------------------------------------------------------------') 

print('-------------------------------80-----------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_80_ft)} (+- {np.std(acc_per_fold_80_ft)})')
print(f'> Loss: {np.mean(loss_per_fold_80_ft)}')
print(f'> F1: {np.mean(f1_fold_80_ft)}(+- {np.std(f1_fold_80_ft)})')
print(f'> Precision: {np.mean(precision_per_fold_80_ft)}(+- {np.std(precision_per_fold_80_ft)})')
print(f'> Recall: {np.mean(recall_per_fold_80_ft)}(+- {np.std(recall_per_fold_80_ft)})')
print('------------------------------------------------------------------------') 

print('-------------------------------40-----------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_40_ft)} (+- {np.std(acc_per_fold_40_ft)})')
print(f'> Loss: {np.mean(loss_per_fold_40_ft)}')
print(f'> F1: {np.mean(f1_fold_40_ft)}(+- {np.std(f1_fold_40_ft)})')
print(f'> Precision: {np.mean(precision_per_fold_40_ft)}(+- {np.std(precision_per_fold_40_ft)})')
print(f'> Recall: {np.mean(recall_per_fold_40_ft)}(+- {np.std(recall_per_fold_40_ft)})')
print('------------------------------------------------------------------------') 

-------------------------------640-----------------------------------------
Average scores for all folds:
> Accuracy: 80.37500023841858 (+- 4.465143155316088)
> Loss: 0.8505165100097656
> F1: 63.107296680647536(+- 2.674772336821832)
> Precision: 63.35769799303158(+- 3.162820665304169)
> Recall: 62.87560576339748(+- 2.305890656121186)
------------------------------------------------------------------------
-------------------------------320-----------------------------------------
Average scores for all folds:
> Accuracy: 80.25000095367432 (+- 3.8242654690409403)
> Loss: 0.8175756096839905
> F1: 63.52477225235627(+- 2.5138350452745457)
> Precision: 64.27871833412178(+- 3.340423843280034)
> Recall: 62.82834463599043(+- 2.085176096070405)
------------------------------------------------------------------------
-------------------------------160-----------------------------------------
Average scores for all folds:
> Accuracy: 80.25000095367432 (+- 3.7207194730947784)
> Loss: 0.78696182966

## Predictions

Finally, we used the fine-tuned model to classify the full set of comments that were not used in the training process or in the experiments

In [9]:
#Predictions of the notAnnotatedComments comments
input_predictions_notAnnotatedComments = df_notAnnotatedComments['review_text']
label_predictions_notAnnotatedComments= df_notAnnotatedComments['label']
predictions_notAnnotatedComments_dataset = tf.data.Dataset.from_generator(
    transform_to_dataset(input_predictions_notAnnotatedComments, label_predictions_notAnnotatedComments),
    output_types=({"word_ids": tf.int64,
                  "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
    output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
predictions_notAnnotatedComments_dataset = predictions_notAnnotatedComments_dataset.batch(32)

predictions_notAnnotatedComments = model.predict(predictions_notAnnotatedComments_dataset)

df_notAnnotatedComments["predict"]= predictions_notAnnotatedComments.argmax(1) 

print('predictions_notAnnotatedComments:', predictions_notAnnotatedComments.shape)
print(df_notAnnotatedComments.predict.value_counts())
print(df_notAnnotatedComments.label.value_counts())

predictions_notAnnotatedComments: (121496, 3)
2    85873
0    25393
1    10230
Name: predict, dtype: int64
2    73514
0    32998
1    14984
Name: label, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
